In [0]:
%pip install openai unidecode gspread==5.12.4 tiktoken mlflow
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
%restart_python
%load_ext autoreload
%autoreload 2 

In [0]:
from abc import ABC, abstractmethod
import time
import mlflow
from contextlib import contextmanager
import json
import pandas as pd
import datetime
import re
import openai
from openai import OpenAI
import gspread
import random
import logging
import os
from unidecode import unidecode
from pathlib import Path
import pyspark
from pyspark.sql.functions import *
from functools import reduce
from typing import *
import tiktoken


from localizers.general_config import *
from localizers.ml_tracker import MLTracker
from localizers.generic_localizer import *
#from InGame_Config import * 

In [0]:
%run "./authenticationScript"

In [0]:
def setup_widgets():
    dbutils.widgets.text('RowFingerprint',"")
    dbutils.widgets.text("Timestamp","")
    dbutils.widgets.text("SubmitterEmail","")
    dbutils.widgets.text("DueDate","")
    dbutils.widgets.text('LocType',"")
    dbutils.widgets.text("Game","")
    dbutils.widgets.text("TargetLanguages","")
    dbutils.widgets.text("URL","")
    dbutils.widgets.text("QAFlag","")
    dbutils.widgets.text("Status","")
    dbutils.widgets.text("LastStatusUpdate","")

def get_request():
    return {'Timestamp':dbutils.widgets.get('Timestamp'), 
       'SubmitterEmail':dbutils.widgets.get('SubmitterEmail'),
       'DueDate': dbutils.widgets.get("DueDate"), 
       'Game':  dbutils.widgets.get("Game"),
       'TargetLanguages':  dbutils.widgets.get("TargetLanguages"), 
       'URL': dbutils.widgets.get("URL"), 
       'LocType': dbutils.widgets.get("LocType"), 
       'RowFingerprint': dbutils.widgets.get("RowFingerprint")}
    
setup_widgets()

In [0]:
#TODO update authenticatinon script and secrets!!
gsheet_client = get_gspread_client_from_secret_old()
gpt_client = get_model_client() 

In [0]:
request = get_request()

# PP cfg
cfg = {
    "input": {"required_tabs": ["input","output"], 
              "input_headers": ['token', 'context', 'en_US'], 
              "output_headers": ['token','en_US','ja','fr',
                'it','de','es','ko','da','sv','no','zh-TW',
                'zh-CN','pt-BR','ru','ar','tr']
    },
    "char_limit_policy": "loose",
    "tracking_sheet_url": "",
}

In [0]:
localizer = GenericLocalizer(request = request, 
                             gsheet_client =gsheet_client, 
                             gpt_client = gpt_client, 
                             cfg=cfg)

# This should be able to handle any localization request that is NOT pubishing or in game, with varying input structures

In [0]:
results = localizer.run()

In [0]:
dbutils.notebook.exit(json.dumps(results))